In [1]:
import csv
import os

import networkx as nx
import nltk
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

In [2]:
ingredients = []
with open('data/ingredients/all_ingredients.txt') as f:
    for line in f:
        ingredient = line.strip()
        ingredients.append(ingredient)
ingredients = sorted(ingredients, key=len, reverse=True)

In [3]:
ingredients[:10]

['high altitude adjustment for deep fat frying',
 "mrs dash's fiesta lime salt-free seasoning",
 'high altitude adjustment for cookie',
 'trappist or monastery-style cheese',
 'high altitude baking quick bread',
 "i can't believe it's not butter",
 'meat: safe cooking temperature',
 'african birdseye chile pepper',
 'heirloom weight & measurement',
 'trompette de la mort mushroom']

In [4]:
techniques = []
with open('data/techniques/all_techniques.txt') as f:
    for line in f:
        technique = line.strip()
        techniques.append(technique)
techniques = sorted(techniques, key=len, reverse=True)

In [5]:
techniques[:10]

['thermal immersion circulator',
 'extreme-heat stir-fry',
 'flash pasteurization',
 'amylolytic process',
 'biomass briquettes',
 'coagulated protein',
 'high heat stir fry',
 'mongolian barbecue',
 'culinary triangle',
 'food preservation']

In [6]:
def singularize(word):
    if word == 'cookies':
        result = 'cookie'
    elif word == 'mrs':
        result = 'mrs'
    elif word == 'ras':
        result = 'ras'
    elif word == 'somen':
        result = 'somen'
    elif word == 'cos':
        result = 'cos'
    elif word == 'monks':
        result = "monk's"
    elif word == 'webbs':
        result = "webb's"
    else:
        result = WordNetLemmatizer().lemmatize(word.lower())
    return result

In [7]:
def inf_ing(v_ing):
    inf = v_ing
    if v_ing == 'de-seeding':
        inf = 'de-seed'
    elif v_ing == 'de-boning':
        inf = 'de-bone'
    elif v_ing == 'de-bearding':
        inf = 'de-beard'
    elif v_ing == 'gutting':
        inf = 'gut'
    elif v_ing == 'de-glazing':
        inf = 'de-glaze'
    elif v_ing == 'sautéing':
        inf = 'sauté'
    elif v_ing == 'degorging':
        inf = 'degorge'
    elif v_ing == 'stir-frying':
        inf = 'stir-fry'
    elif v_ing == 'broasting':
        inf = 'broast'
    elif v_ing == 'juicing':
        inf = 'juice'
    elif v_ing == 'parbaking':
        inf = 'parbake'
    elif v_ing == 'charbroiling':
        inf = 'charbroil'
    elif v_ing == 'crinkle-cutting':
        inf = 'crinkle-cut'
    elif v_ing == 'cheesemaking':
        inf = 'cheesemake'
    elif v_ing == 'swissing':
        inf = 'swiss'
    elif v_ing == 'flash-frying':
        inf = 'flash-fry'
    elif v_ing == 'flashbaking':
        inf = 'flashbake'
    else:
        inf = WordNetLemmatizer().lemmatize(v_ing.lower(), 'v')
    return inf

def infinitive(words):
    aux = []
    word_list = words.split()
    for word in word_list:
        if word != 'cook' and word != 'cooking':
            if word.endswith('ing'):
                inf = inf_ing(word.lower())
                aux.append(inf)
#                 if inf == word:
#                     print('NO INFINITIVE FOR', word)
            else:
                aux.append(word.lower())
    result = ' '.join(aux)
    return result

In [8]:
ingr_id_graph = nx.read_gexf('data/ingredients/ingr_id_graph.gexf')

In [10]:
folder_name

'111000'

In [12]:
sorted(map(int, os.listdir(path)))[111]

111000

In [19]:
recipe_info['data-title'][1:-2].encode('utf_8').decode('unicode_escape')

UnicodeDecodeError: 'unicodeescape' codec can't decode byte 0x5c in position 30: \ at end of string

In [20]:
with open('data/ingredients/allrecipes_ingr_ids_111.csv', 'w') as f1,\
     open('data/techniques/allrecipes_techniques_111.csv', 'w') as f2,\
     open('data/ingredient_technique_log_111.txt', 'w') as log:
    writer1 = csv.writer(
        f1,
        delimiter=',',
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL
    )
    writer2 = csv.writer(
        f2,
        delimiter=',',
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL
    )
    writer3 = csv.writer(
        log,
        delimiter=',',
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL
    )
    path = '/media/antonio/WD1T/datasets allrecipes and epicurious/allrecipes/'
#     path = 'data/allrecipes/'
    for folder_number in sorted(map(int, os.listdir(path)))[111:]:
        folder_name = str(folder_number)
        folder_path = path + folder_name
        for file_name in sorted(map(lambda x: int(x[:-5]), os.listdir(folder_path))):
            file_name = str(file_name) + '.html'
            file_path = folder_path + '/' + file_name
            with open(file_path) as f:
                soup = BeautifulSoup(f.read(), 'html.parser')
                recipe_info = soup.find('recipe-signup')
                recipe_id = recipe_info['data-id']
                recipe_title = recipe_info['data-title'][1:-1].encode('utf_8').decode('unicode_escape')
                info_row = [recipe_id, recipe_title]
                
                ingredient_ids = set()
                ingredient_amounts = soup.select('section.recipe-ingredients span[itemprop=ingredients]')
                for ingredient_amount in ingredient_amounts:
                    ingredient_id = ingredient_amount['data-id']
                    content = ingredient_amount.text.strip()
                    if content:
                        tokens = nltk.word_tokenize(content)
                        singularized = ' '.join(map(singularize, tokens))
                        found_ingr = False
                        for ingredient in ingredients:
                            if ingredient in singularized:
                                found_ingr = True
                                ingredient_ids.add(ingredient_id)
                                if ingr_id_graph.add_edge(ingredient_id, ingredient):
                                    ingr_id_graph[ingredient_id][ingredient]['weight'] += 1
                                else:
                                    ingr_id_graph.add_edge(ingredient_id, ingredient, weight=1)
                        if not found_ingr:
                            writer3.writerow(['INGREDIENT', file_path, singularized])
                ingredient_row = info_row + list(ingredient_ids)
                writer1.writerow(ingredient_row)
                
                technique_names = set()
                instructions = soup.select('section.recipe-directions span.recipe-directions__list--item')
                for instruction in instructions:
                    content = instruction.text.strip()
                    if content:
                        tokens = nltk.word_tokenize(content)
                        infinitived = ' '.join(map(infinitive, tokens))
                        found_tech = False
                        for technique in techniques:
                            if technique in infinitived:
                                found_tech = True
                                technique_names.add(technique)
                        if not found_tech:
                            writer3.writerow(['TECHNIQUE', file_path, infinitived])
                technique_row = info_row + list(technique_names)
                writer2.writerow(technique_row)

In [8]:
# ingr_id_graph = nx.Graph()
path = '/media/antonio/WD1T/datasets allrecipes and epicurious/allrecipes/'
#     path = 'data/allrecipes/'
c=0
ingredient_ids = set()
for folder_number in sorted(map(int, os.listdir(path))):
    folder_name = str(folder_number)
    folder_path = path + folder_name
    for file_name in sorted(map(lambda x: int(x[:-5]), os.listdir(folder_path))):
        file_name = str(file_name) + '.html'
        file_path = folder_path + '/' + file_name
        with open(file_path) as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
#             recipe_info = soup.find('recipe-signup')
#             recipe_id = recipe_info['data-id']
#             recipe_title = recipe_info['data-title'][1:-1].encode('utf_8').decode('unicode_escape')
#             info_row = [recipe_id, recipe_title]

            ingredient_amounts = soup.select('section.recipe-ingredients span[itemprop=ingredients]')
            for ingredient_amount in ingredient_amounts:
                ingredient_id = ingredient_amount['data-id']
                content = ingredient_amount.text.strip()
                if content:
                    tokens = nltk.word_tokenize(content)
                    singularized = ' '.join(map(singularize, tokens))
#                     found_ingr = False
                    for ingredient in ingredients:
                        if ingredient in singularized:
#                             found_ingr = True
                            ingredient_ids.add(ingredient_id)
#                             if ingr_id_graph.has_edge(ingredient_id, ingredient):
#                                 ingr_id_graph[ingredient_id][ingredient]['weight'] += 1
#                             else:
#                                 ingr_id_graph.add_edge(ingredient_id, ingredient, weight=1)
#                     if not found_ingr:
#                         writer3.writerow(['INGREDIENT', file_path, singularized])
#             ingredient_row = info_row + list(ingredient_ids)
#             writer1.writerow(ingredient_row)
            c+=1
            if c % 10000 == 0:
                print(c, 'recipes processed.')
# nx.write_gexf(ingr_id_graph, 'data/ingredients/ingr_id_graph.gexf')
print(len(ingredients_ids))

10000 recipes processed.
20000 recipes processed.
30000 recipes processed.
40000 recipes processed.
50000 recipes processed.
60000 recipes processed.
70000 recipes processed.
80000 recipes processed.
90000 recipes processed.
100000 recipes processed.
110000 recipes processed.
120000 recipes processed.
130000 recipes processed.
140000 recipes processed.
150000 recipes processed.
160000 recipes processed.
170000 recipes processed.
180000 recipes processed.
190000 recipes processed.
200000 recipes processed.
210000 recipes processed.


NameError: name 'ingredients_ids' is not defined

In [16]:
print(len(ingredient_ids))

NameError: name 'ingredient_ids' is not defined

In [ ]:
Store ingredient_ids on a pickle object

In [9]:
ingr_id_graph.number_of_nodes()

12242

In [10]:
ingr_id_graph.number_of_edges()

43837

In [11]:
c=0
for n in ingr_id_graph.nodes(data=False):
#     ns = ingr_id_graph.neighbors(n)
#     for ne in ns:
    anyy=False
    for ne in ingr_id_graph[n]:
        if ingr_id_graph[n][ne]['weight'] > 1:
#             print(n)
#             print(ingr_id_graph[n])
#             print()
            anyy=True
            break
    if anyy:
        c+=1
c

8263

In [15]:
ingrs = set()
for n in ingr_id_graph.nodes(data=False):
#     ns = ingr_id_graph.neighbors(n)
#     for ne in ns:
    for ne in ingr_id_graph[n]:
        if ingr_id_graph[n][ne]['weight'] > 1:
#             print(n)
#             print(ingr_id_graph[n])
#             print()
            anyy=True
            break
    if anyy:
        c+=1
c

In [15]:
qwe=1

In [12]:
c=0
nums=set()
for n in ingr_id_graph.nodes(data=False):
#     ns = ingr_id_graph.neighbors(n)
#     for ne in ns:
    
    if not n.isdigit():
        c+=1
        nums.add(n)
c

1382

In [35]:
nums2=set()
for n in ingr_id_graph.nodes(data=False):
#     ns = ingr_id_graph.neighbors(n)
#     for ne in ns:
    
    if n.isdigit():
        ns = ingr_id_graph.neighbors(n)
        for ne in ns:
            if ne.isdigit():
                print(n, ne)
            nums2.add(ne)
#         print(ingr_id_graph[n])
#         break


In [36]:
len(nums2)

1382

In [21]:
list(nums2)[:10]

['21046',
 '4620',
 '24098',
 '2159',
 '1606',
 '21771',
 '25719',
 '4617',
 '23688',
 '21572']

In [13]:
len(nums)

1382

In [ ]:
# nx.write_gexf(ingr_id_graph, 'data/ingredients/ingr_id_graph.gexf')

In [ ]:
# with open('data/allrecipes/6000/6903.html') as f:
#     soup = BeautifulSoup(f.read(), 'html.parser')
#     info = soup.find('recipe-signup')
#     print(info['data-id'])
#     print(info['data-title'])
#     qq=info['data-title']
#     info_row = [info['data-id'], info['data-title']]
#     technique_names = []
#     instructions = soup.select('section.recipe-directions span.recipe-directions__list--item')
#     for instruction in instructions:
#         infinitived = ' '.join(map(infinitive, instruction.string.strip().split()))
#         for technique in techniques:
#             if technique in infinitived:
#                 technique_names.append(technique)
#     technique_row = info_row + technique_names
#     print(technique_row)
#     print()

In [ ]:
# las tecnicas no tienen pq ser solo verbos - pueden ser sustantivos, por ejemplo, bbq, fondue...

In [ ]:
# for folder_name in os.listdir('data/allrecipes/'):
#     folder_path = 'data/allrecipes/' + folder_name
#     for file_name in os.listdir(folder_path):
#         file_path = folder_path + '/' + file_name
#         with open(file_path) as f:
#             soup = BeautifulSoup(f.read(), 'html.parser')
#             info = soup.find('recipe-signup')
#             print(info['data-id'])
#             print(info['data-title'])
#             ingredients = soup.select('span[itemprop=ingredients]')
#             for ingredient in ingredients:
#                 print(ingredient.string.strip())
#             print()